In [3]:
import os
from time import time

import openai
from dotenv import load_dotenv, find_dotenv
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(temperature=0)

prompt = ChatPromptTemplate.from_template(
    'Classify the text based on the Common European Framework of Reference '
    'for Languages (CEFR). Give a single value: {text}',
)
chain = LLMChain(llm=llm, prompt=prompt)

texts = [
    {'text': 'Hallo, ich bin 25 Jahre alt.'},
    {'text': 'Wie geht es dir?'},
    {'text': 'In meiner Freizeit, spiele ich gerne Fussball.'}
]

start = time()
res_a = chain.apply(texts)
print(res_a)
print(f"apply time taken: {time() - start:.2f} seconds")
print()

start = time()
res_aa = await chain.aapply(texts)
print(res_aa)
print(f"aapply time taken: {time() - start:.2f} seconds")

[{'text': 'Based on the given text "Hallo, ich bin 25 Jahre alt," it can be classified as CEFR level A1.'}, {'text': 'A2'}, {'text': 'A2'}]
apply time taken: 2.87 seconds

[{'text': 'Based on the given text "Hallo, ich bin 25 Jahre alt," it can be classified as CEFR level A1.'}, {'text': 'A2'}, {'text': 'A2'}]
aapply time taken: 1.34 seconds


In [6]:
import openai
import os
from time import time

from pprint import pprint

import pandas as pd
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
df = pd.read_csv('../data/product_reviews.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


# Langy

### Classify Level of Text and Provide Reason

In [29]:
llm = ChatOpenAI(temperature=0)

cefr_template ="""Classify the text based on the Common European Framework of Reference
for Languages (CEFR), provide a reason for your answer.

Text: {text}

{format_instructions}
 """

prompt = ChatPromptTemplate.from_template(cefr_template)

chain = LLMChain(llm=llm, prompt=prompt)

# Define output schema
level_schema = ResponseSchema(name="level",
                             description="The CEFR level of the text, e.g. A1, B2, etc.")
reason_schema = ResponseSchema(name="reason",
                                    description="The reason for the classification")

response_schemas = [level_schema, reason_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

text = """Hallo, ich heisse Adam. Ich habe 25 Jahre alt. Ich wohne in England seit 15 Jahren aber ich wuerde gerne irgendwo anders wohnen. Ich liebe es zu reisen. Meiner Meinung nach, ist man wirklich am leben, wenn man reist."""

# messages = prompt.format(text=text, format_instructions=format_instructions)

# messages = [
#     {'text': text, 'format_instructions': format_instructions}
# ]
# response = chain.apply(messages)

messages = {'text': text, 'format_instructions': format_instructions}
response = chain.run(messages)

result = output_parser.parse(response)
print(result)
type(result)
# response[0]['text']

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"level": string  // The CEFR level of the text, e.g. A1, B2, etc.
	"reason": string  // The reason for the classification
}
```
{'level': 'A2', 'reason': 'The text demonstrates a basic understanding of the language with simple sentence structures and limited vocabulary. The use of present tense and basic expressions indicate a beginner level of proficiency.'}


dict

### Correct Grammar and Spelling

In [32]:
llm = ChatOpenAI(temperature=0)

gram_spell_template ="""Correct the grammar and spelling of the text. Provide a reason for
each correction.

Text: {text}

{format_instructions}
 """

prompt = ChatPromptTemplate.from_template(gram_spell_template)

chain = LLMChain(llm=llm, prompt=prompt)

# Define output schema
corrected_schema = ResponseSchema(name="corrected_text",
                                    description="The corrected text")
reasons_schema = ResponseSchema(name="reasons",
                                description="The reasons for the corrections")


response_schemas = [corrected_schema, reasons_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

text = """Hallo, ich heisse Adam. Ich habe 25 Jahre alt. Ich wohne in England seit 15 Jahren aber ich wuerde gerne irgendwo anders wohnen. Ich liebe es zu reisen. Meiner Meinung nach, ist man wirklich am leben, wenn man reist."""

messages = {
    'text': text,
    'format_instructions': format_instructions
}
response = chain.run(messages)
# response

result = output_parser.parse(response)
print(result)
type(result)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"corrected_text": string  // The corrected text
	"reasons": string  // The reasons for the corrections
}
```
{'corrected_text': 'Hallo, ich heiße Adam. Ich bin 25 Jahre alt. Ich wohne seit 15 Jahren in England, aber ich würde gerne irgendwo anders wohnen. Ich liebe es zu reisen. Meiner Meinung nach ist man wirklich am Leben, wenn man reist.', 'reasons': "1. Corrected the spelling of 'heisse' to 'heiße' because it should have an 'ß' instead of 'ss'.\n2. Changed 'habe' to 'bin' because '25 Jahre alt' is a state of being, not possession.\n3. Added a comma after 'England' to separate the clauses.\n4. Removed the space before 'wuerde' to correct the spelling.\n5. Removed the space before 'Meiner' to correct the sentence structure.\n6. Removed the comma after 'nach' to improve the flow of the sentence."}


dict

In [5]:
texts = [
    {'text': 'Hallo, ich bin 25 Jahre alt.'},
    {'text': 'Wie geht es dir?'},
    {'text': 'In meiner Freizeit spiele ich gerne Fussball.'}
]
start = time()
res_a = chain.apply(texts)
print(res_a)
print(f"apply time taken: {time() - start:.2f} seconds")
print()

start = time()
res_aa = await chain.aapply(texts)
print(res_aa)
print(f"aapply time taken: {time() - start:.2f} seconds")

[{'text': 'Based on the provided text "Hallo, ich bin 25 Jahre alt," it can be classified as A1 level according to the Common European Framework of Reference for Languages (CEFR). \n\nThe reason for this classification is that the text consists of simple and basic vocabulary and sentence structures. It includes a greeting ("Hallo") and a simple statement about the person\'s age ("ich bin 25 Jahre alt"). A1 level is the beginner level, where learners can understand and use familiar everyday expressions and basic phrases to communicate simple and immediate needs.'}, {'text': 'According to the Common European Framework of Reference for Languages (CEFR), the text "Wie geht es dir?" can be classified as A1 level. \n\nThe reason for this classification is that the text consists of a simple question in German, asking "How are you?" This type of question is commonly taught at the beginner level, where learners are introduced to basic greetings and expressions. At the A1 level, learners are exp

In [11]:
chain.predict(a=texts[0])

ValueError: Missing some input keys: {'text'}

# LLMChain

In [14]:
# High temp -> more creative names
llm = ChatOpenAI(temperature=0.9)
prompt = ChatPromptTemplate.from_template(
    'What is the best name to describe a company that sells {product}?',
)
chain = LLMChain(llm=llm, prompt=prompt)

for product in df.Product.iloc[:5]:
    company_name_idea = chain.run(product)
    print(f"Product: {product}, Generated name: {company_name_idea}")

Product: Queen Size Sheet Set, Generated name: "Royal Dreams Linens" or "Regal Comfort Bedding"
Product: Waterproof Phone Pouch, Generated name: Aquashield
Product: Luxury Air Mattress, Generated name: 1. Opulent Rest
2. LuxeSlumber
3. Supreme Sleep
4. Elite Air Mattress Co.
5. DreamSerenity
6. LuxAir Mattress
7. PlushDreams
8. PrestigeSleep
9. RegalAir Mattress
10. SereneSlumber
Product: Pillows Insert, Generated name: The Cushion Corner
Product: Milk Frother Handheld
, Generated name: "Milk Frother Plus" or "FrothMaster"


In [20]:
# You can also generate multiple results at once like this
products = list(df.Product.iloc[:5])
products = [{'product': product} for product in products]
chain.generate(products)

LLMResult(generations=[[ChatGeneration(text='RoyalComfort Beddings', generation_info=None, message=AIMessage(content='RoyalComfort Beddings', additional_kwargs={}, example=False))], [ChatGeneration(text='AquaGuard Phone Accessories', generation_info=None, message=AIMessage(content='AquaGuard Phone Accessories', additional_kwargs={}, example=False))], [ChatGeneration(text='OpulentRest', generation_info=None, message=AIMessage(content='OpulentRest', additional_kwargs={}, example=False))], [ChatGeneration(text='PillowPerfect', generation_info=None, message=AIMessage(content='PillowPerfect', additional_kwargs={}, example=False))], [ChatGeneration(text='FrothMaster', generation_info=None, message=AIMessage(content='FrothMaster', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 116, 'completion_tokens': 19, 'total_tokens': 135}, 'model_name': 'gpt-3.5-turbo'})

Note: you can use OpenAI via your Azure deployment like so

```python
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
deployment_name="your-azure-deployment",
model_name="text-davinci-003"
)
```

# SimpleSequentialChain

The simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.

In [12]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    'What is the best name to describe a company that sells {product}?',
)
first_chain = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(
    'Write a 20 word description for the following company: {company_name}',
)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(
    chains=[first_chain, second_chain],
    verbose=True,
)

for product in df.Product.iloc[:5]:
    overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Comfort Beddings
Royal Comfort Beddings is a luxury bedding brand offering top-quality, stylish bedding sets and accessories for a blissful sleep experience.

> Finished chain.


> Entering new SimpleSequentialChain chain...
AquaGuard Phone Protection
AquaGuard Phone Protection offers high-quality waterproof cases and screen protectors to ensure maximum protection for your valuable smartphones.

> Finished chain.


> Entering new SimpleSequentialChain chain...
Opulent Rest
Opulent Rest offers luxurious and high-quality bedding products that provide the ultimate comfort and relaxation for a restful night's sleep.

> Finished chain.


> Entering new SimpleSequentialChain chain...
PillowPro
PillowPro offers premium and customizable pillows designed for optimal comfort and support, ensuring a restful night's sleep.

> Finished chain.


> Entering new SimpleSequentialChain chain...
FrothMaster
FrothMaster is a leading manufacturer of hig

# SequentialChain

A more general form of sequential chains, allowing for multiple inputs/outputs.

In [ ]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    'Translate the following review to English: {review}',
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key='english_review')

second_prompt = ChatPromptTemplate.from_template(
    'Summarize the following review in 1 sentence: {english_review}',
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key='summary')

third_prompt = ChatPromptTemplate.from_template(
    'What language is the following review: {review}'
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key='language')

fourth_prompt = ChatPromptTemplate.from_template(
    'What a follow up response to the following summary in the specified'
    'language: Summary: {summary}\n\n Language: {language}'
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key='follow_up_response')

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['review'],
    output_variables=['english_review', 'summary', 'follow_up_response'],
    verbose=True,
)